In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
import statsmodels.api as sm
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, confusion_matrix, log_loss
from sklearn.metrics import precision_recall_curve, roc_curve
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.preprocessing import QuantileTransformer, PowerTransformer
from sklearn.feature_selection import SelectKBest, SelectFromModel, RFE, RFECV, SelectPercentile, SelectFpr, SelectFdr, SelectFwe
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif, SelectFdr
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
import matplotlib.pyplot as plt
import pickle
from sklearn.decomposition import PCA
# import lifelines as ll
# from lifelines.utils.sklearn_adapter import sklearn_adapter
# CoxRegression = sklearn_adapter(ll.CoxPHFitter, event_col = 'event')
import sys
sys.path.append('/odinn/users/thjodbjorge/Python_functions/')
import Predict_functions as pf
from Calculate_score import calculate_metrics

In [ ]:
folder = '/odinn/users/thjodbjorge/Proteomics/Mortality2/'
feat_folder = 'Features2/'
pred_folder = 'Predictions4/'
# corr_type = 'sitesampleageqt'
corr_type = 'None'


In [ ]:
if corr_type == 'qt':
    print('Load qt transformed proteins')
    proteins = pd.read_csv('/odinn/users/thjodbjorge/Proteomics/Data/protein_data/protein_qt.csv',index_col = 'Barcode2d' )
else:
    print('Load raw protein values')
    raw_data = pd.read_csv('/odinn/users/thjodbjorge/Proteomics/Data/raw_with_info.csv',index_col = 'Barcode2d' )
    
probe_info = pd.read_csv('/odinn/users/thjodbjorge/Proteomics/Data/probe_info.csv', index_col = 'SeqId')

pn_info = pd.read_csv('/odinn/users/thjodbjorge/Proteomics/Data/pn_info_Mor/pn_info_Mor_event.csv',index_col = 'Barcode2d' )
probes_to_skip = pd.read_csv('/odinn/users/thjodbjorge/Proteomics/Data/probes_to_skip.txt')['probe']

In [ ]:

if corr_type == 'pqtl':
    pqtl_protein = pd.read_csv('/odinn/users/egilf/pQTL/for_benedikt/pQTL_conditional_04052020.gor', sep='\t')
    # pqtl = pd.read_csv('/odinn/users/steinthora/proteomics/proteomic_project/Data/pQTL_Merged_08052020.csv', sep = '\t', index_col = 'PN')
    pqtl = pd.read_csv('/odinn/users/thjodbjorge/Proteomics/pqtl/pqtl_combined_meanimp.csv',index_col = 'PN')


    # In[4]:

    pqtl = pd.merge(pn_info['PN'],pqtl,left_on='PN',right_index=True)
    pqtl.drop('PN',axis=1,inplace=True)
    pro_pqtl = {}
    for i in raw_data.iloc[:,16:].columns:
        pro_pqtl[i] = list(pqtl_protein[pqtl_protein.SeqId == i[6:].replace('-','_')]['SentinelMarker'])

In [ ]:
endpoints = ['death']
# endpoints = ['death','Cdeath','Gdeath','Ideath','Jdeath','Otherdeath']
# event_date = event_date_death
time_to_event = pn_info.time_to_death
no_event_before = pn_info.no_death_before
for endpoint in endpoints:
    if endpoint == 'death':
        use_event = pn_info.event_death
        print(use_event.sum())
    elif endpoint == 'Cdeath':
        use_event = pn_info.event_death & (pn_info.ICD_group == 'C')
        print(use_event.sum())
    elif endpoint == 'Gdeath':
        use_event = pn_info.event_death & (pn_info.ICD_group == 'G')
        print(use_event.sum())
    elif endpoint == 'Ideath':
        use_event = pn_info.event_death & (pn_info.ICD_group == 'I')
        print(use_event.sum())
    elif endpoint == 'Jdeath':
        use_event = pn_info.event_death & (pn_info.ICD_group == 'J')
        print(use_event.sum())
    elif endpoint == 'Otherdeath':
        use_event = pn_info.event_death & (~(pn_info.ICD_group == 'C')&~(pn_info.ICD_group == 'G')&~(pn_info.ICD_group == 'I')&~(pn_info.ICD_group == 'J'))
        print(use_event.sum())

y = []
for i in range(1,19):
    y.append(use_event & (time_to_event <= i))

kf = KFold(n_splits=10, random_state=10, shuffle=False) 
I_train_main, I_test_main = train_test_split(pn_info.index, train_size=0.7, random_state = 10)
# I_val_main, I_test_main = train_test_split(I_test_main, train_size=0.5, random_state = 10)



file = open(folder+"{}_keep_samples.pkl".format('Mor'),'rb')
keep_samples_dict = pickle.load(file)

# print(keep_samples_dict.keys())
# keep_samples_keys = ['Old_18105', 'Old_60105', 'Old_6080','Old_18105_C', 'Old_18105_I', 'Old_18105_J', 'Old_18105_G','Old_18105_Other']
keep_samples_keys = ['Old_18105']#,'Old_60105']
# keep_samples_keys = ['Old_no_comor_18105']
# keep_samples_keys = ['Old_18105_Neoplasms','Old_18105_I','Old_18105_J','Old_18105_G','Old_18105_Other']
# keep_samples_keys = ['Old_cardio_risk_18105']
skip_PC = 5

In [ ]:
for dataset in keep_samples_keys:

    print(dataset)
    keep_samples = keep_samples_dict[dataset]

    I_train = I_train_main.intersection(keep_samples)#.intersection(have_prs)
    I_test = I_test_main.intersection(keep_samples)#.intersection(have_prs)

    print('Training set: {}, MI within 15: {}, 10: {}, 5: {}, 2: {}'.format(len(I_train),y[14][I_train].sum(),y[9][I_train].sum(),y[4][I_train].sum(),y[1][I_train].sum()))
    print('Test set: {}, MI within 15: {}, 10: {}, 5: {}, 2: {}'.format(len(I_test),y[14][I_test].sum(),y[9][I_test].sum(),y[4][I_test].sum(),y[1][I_test].sum()))

        # ### Select data and normalize

    if corr_type == 'qt':
        X = proteins
    else:
        X = np.log(raw_data.iloc[:,16:].drop(probes_to_skip,axis=1))

    all_protein = X.columns
    X['sex'] = pn_info[['sex']].values-1
    X['age'] = pn_info[['Age_at_sample_collection_2']].values

    X['age2'] = X['age']**2
#     X['age3'] = X['age']**3
    X['agesex'] = X['age']*X['sex']
    X['age2sex'] = X['age2']*X['sex']
        
    agesex = ['age','sex','agesex','age2','age2sex']
       
    X['lnage'] = np.log(X['age'])
    X['lnage2'] = X['lnage']**2
    
    X['PAD'] = pn_info['PAD']
    no_bmi = (X['PAD'].isna())
    no_bmi_ind = X[no_bmi].index
    X.loc[I_train.intersection(no_bmi_ind),'PAD'] = X.loc[I_train].PAD.mean()
    X.loc[I_test.intersection(no_bmi_ind),'PAD'] = X.loc[I_test].PAD.mean()
    
    
    X['CAD'] = ~pn_info.no_CAD_before
    X['MI'] = ~pn_info.no_MI_before
    X['PCE'] = ~pn_info.no_PCEend_before
    X['Stroke'] = ~pn_info.no_stroke_before
    X['cancer'] = pn_info.cancer_margin
    X['ApoB'] = X['SeqId.2797-56']
    X['Smoker'] = pn_info['Smoker'].astype(int).values
    X['diabetes'] = pn_info['T2D'].astype(int).values
    X['HTN_treated'] = pn_info[['HTN_treated']].astype(int).values
#     X['statin'] = pn_info['statin'].astype(int).values
    X['statin'] = pn_info['statin_estimate_unsure'].astype(int).values
    X['ApoBstatin']  = X['ApoB']*X['statin']

    X['cancer1y']  = pn_info['cancer1y']
    X['cancer5y']  = pn_info['cancer5y']

    X['GDF15'] = X['SeqId.4374-45'].copy()
    X['GDF152'] = X['GDF15']**2
    X['GDF15age']  = X['GDF15']*X['age']
    X['GDF15sex']  = X['GDF15']*X['sex']
    
    X['bmi'] = pn_info['bmi']

    no_bmi = (X['bmi'].isna())
    no_bmi_ind = X[no_bmi].index
#     X.loc[no_bmi_ind,'bmi'] = X.loc[I_train].bmi.mean()
    X.loc[I_train.intersection(no_bmi_ind),'bmi'] = X.loc[I_train].bmi.mean()       
    X.loc[I_test.intersection(no_bmi_ind),'bmi'] = X.loc[I_test].bmi.mean()   
    
    X['bmi2'] = X['bmi']*X['bmi']
    
    X['Platelets'] = pn_info['Platelets']
    no_p = (X['Platelets'].isna()); print(no_p.sum())
    no_p_ind = X[no_p].index
    X.loc[I_train.intersection(no_p_ind),'Platelets'] = X.loc[I_train].Platelets.mean()
    X.loc[I_test.intersection(no_p_ind),'Platelets'] = X.loc[I_test].Platelets.mean()
    X['Platelets2'] = X['Platelets']*X['Platelets']
    
    X['Creatinine'] = pn_info['Creatinine']
    no_p = (X['Creatinine'].isna()); print(no_p.sum())
    no_p_ind = X[no_p].index
    X.loc[I_train.intersection(no_p_ind),'Creatinine'] = X.loc[I_train].Creatinine.mean()
    X.loc[I_test.intersection(no_p_ind),'Creatinine'] = X.loc[I_test].Creatinine.mean()
    
    X['Triglycerides'] = pn_info['Triglycerides']
    no_p = (X['Triglycerides'].isna()); print(no_p.sum())
    no_p_ind = X[no_p].index
    X.loc[I_train.intersection(no_p_ind),'Triglycerides'] = X.loc[I_train].Triglycerides.mean()    
    X.loc[I_test.intersection(no_p_ind),'Triglycerides'] = X.loc[I_test].Triglycerides.mean()   
    

    X['bmiage'] = X['bmi']*X['age']
    X['bmisex'] = X['bmi']*X['sex']
    X['bmi2age'] = X['bmi2']*X['age']
    X['bmi2sex'] = X['bmi2']*X['sex']
    X['PADage'] = X['PAD']*X['age']
    X['PADsex'] = X['PAD']*X['sex']
    
    X['ApoBage']  = X['ApoB']*X['age']
    X['Smokerage'] = X['Smoker']*X['age']
    X['diabetesage'] = X['diabetes']*X['age'] 
    X['statinage'] = X['statin']*X['age']
    X['CADage'] = X['CAD']*X['age']
    X['MIage'] = X['MI'] * X['age']
    X['HTN_treatedage'] =  X['age']*X['HTN_treated']    
    X['cancerage'] = X['age']*X['cancer']
    
    X['Plateletsage'] = X['Platelets']*X['age']
    X['Creatinineage'] = X['Creatinine']*X['age']
    X['Triglyceridesage'] = X['Triglycerides']*X['age']    
    X['Platelets2age'] = X['Platelets2']*X['age']
    
    X['cancer1yage'] = X['cancer1y']*X['age'] 
    X['cancer5yage'] = X['cancer5y']*X['age']     
    
    X['ApoBsex']  = X['ApoB']*X['sex']
    X['Smokersex'] = X['Smoker']*X['sex']
    X['diabetessex'] = X['diabetes']*X['sex'] 
    X['statinsex'] = X['statin']*X['sex']
    X['CADsex'] = X['CAD']*X['sex']
    X['MIsex'] = X['MI'] * X['sex']
    X['HTN_treatedsex'] =  X['sex']*X['HTN_treated']   
    X['cancersex'] = X['sex']*X['cancer']
    
    X['Plateletssex'] = X['Platelets']*X['sex']
    X['Creatininesex'] = X['Creatinine']*X['sex']
    X['Triglyceridessex'] = X['Triglycerides']*X['sex']        
    
    X = X.join(pd.get_dummies(pn_info['agebin'],drop_first = True,prefix='age'))
    X['ageage2'] = X['age']*X['age_2.0']
    X['ageage3'] = X['age']*X['age_3.0']
    X['ageage4'] = X['age']*X['age_4.0']
    
    agebins = ['age_2.0','age_3.0','age_4.0', 'ageage2','ageage3','ageage4']
    agebinssex = [s+'sex' for s in agebins]
    X[agebinssex] = (X[agebins].transpose()*X['sex']).transpose()    
    
    batch_var = pd.get_dummies(pn_info['batch'],drop_first = True).columns
    X[batch_var] = pd.get_dummies(pn_info['batch'],drop_first = True)
    
    PRS = ['nonHDL_prs', 'HT_prs', 'CAD_prs', 'Cancer_prs', 'Stroke2_prs', 'alz_Jansen',
       'pgc_adhd_2017', 'PD_Nalls_2018', 'edu_160125', 'dep_2018', 'bpd_2018',
       'giant_bmi', 'schizo_clozuk', 'iq_2018', 'ipsych_pgc_aut_2017',
       'pgc_Anorexia_2019']
    X[PRS] = pn_info[PRS]
    
    X['site'] = (pn_info['site'] == 'DC').astype(int)
    X['Sample_age'] = pn_info['Sample_age']
    
    try: 
        print('Load age dictonary')
        file = open(folder+pred_folder+"age_predict.pkl",'rb')
        age_dict = pickle.load(file)
        file.close()
        PAD2 = age_dict['{}_sexprotein_lasso'.format(dataset)][4]-X.age
        X['PAD2'] = PAD2
    except:
        print('No file to load')
    if corr_type == 'pqtl':
        X =X.merge(pqtl,how = 'left', right_index=True, left_index=True)
    
    X_train = X.loc[I_train]
    X_test = X.loc[I_test]
    
    if corr_type == 'pqtl':
        ### Correct for pqtls
        for m in pqtl.columns:
            no_p = (X[m].isna());# print(no_p.sum())
            no_p_ind = X[no_p].index
            X_train.loc[I_train.intersection(no_p_ind),m] = X_train[m].mean()
            X_test.loc[I_test.intersection(no_p_ind),m] = X_test[m].mean()

        for p in all_protein:    
            pqtl_model = sm.OLS(X_train[p],sm.add_constant(X_train[pro_pqtl[p]])).fit()
            corr_train = pqtl_model.predict(sm.add_constant(X_train[pro_pqtl[p]]))
            corr_test = pqtl_model.predict(sm.add_constant(X_test[pro_pqtl[p]]))
        #     corr_train.columns = all_protein
            X_train[p] = X_train[p] - corr_train
            X_test[p] = X_test[p] - corr_test
            
    if corr_type == 'sitesampleage':
        print('Correct proteins fro site and sample age')
        for p in all_protein:    
            corr_model = sm.OLS(X_train[p],sm.add_constant(X_train[['site','Sample_age']])).fit()
            corr_train = corr_model.predict(sm.add_constant(X_train[['site','Sample_age']]))
            corr_test = corr_model.predict(sm.add_constant(X_test[['site','Sample_age']]))
        #     corr_train.columns = all_protein
            X_train[p] = X_train[p] - corr_train
            X_test[p] = X_test[p] - corr_test
        print('Correction done')     
        
        
    if corr_type == 'sitesampleageqt':
        print('Correct proteins for site and sample age')
        for p in all_protein:    
            corr_model = sm.OLS(X_train[p],sm.add_constant(X_train[['site','Sample_age']])).fit()
            corr_train = corr_model.predict(sm.add_constant(X_train[['site','Sample_age']]))
            corr_test = corr_model.predict(sm.add_constant(X_test[['site','Sample_age']]))
        #     corr_train.columns = all_protein
            X_train[p] = X_train[p] - corr_train
            X_test[p] = X_test[p] - corr_test
        print('Normalize proteins')    
        transformer = QuantileTransformer(n_quantiles=50000, output_distribution = 'normal',random_state=10)
        transformer.fit(X_train[all_protein])
        X_train[all_protein] = transformer.transform(X_train[all_protein])
        X_test[all_protein] = transformer.transform(X_test[all_protein]) 
        
    if corr_type == 'batch':
        for p in all_protein:    
            corr_model = sm.OLS(X_train[p],sm.add_constant(X_train[batch_var])).fit()
            corr_train = corr_model.predict(sm.add_constant(X_train[batch_var]))
            corr_test = corr_model.predict(sm.add_constant(X_test[batch_var]))
        #     corr_train.columns = all_protein
            X_train[p] = X_train[p] - corr_train
            X_test[p] = X_test[p] - corr_test           
        
    
    if corr_type == 'PCA':
        pca1 = PCA(skip_PC)
        x_pca1 = pca1.fit_transform(X_train[all_protein])
        x_1 = pca1.inverse_transform(x_pca1)
        X_train[all_protein] = X_train[all_protein] - x_1

        x_pca1 = pca1.transform(X_test[all_protein])
        x_1 = pca1.inverse_transform(x_pca1)
        X_test[all_protein] = X_test[all_protein] - x_1
    
    
    train_mean = X_train.mean()
    train_std = X_train.std()

    X_train = (X_train-train_mean)/train_std
    X_test = (X_test-train_mean)/train_std

        ## For survival analysis    
    X_train['event'] = use_event[I_train]
    X_test['event'] = use_event[I_test]

    tte_train = time_to_event[I_train]
    tte_test = time_to_event[I_test]

    ysurv_train = pd.DataFrame()
    ysurv_train['event'] = use_event[I_train]
    ysurv_train['time_to_event'] = time_to_event[I_train]
    print('Done')

In [ ]:
print(corr_type)
if corr_type == 'qt':
    print('qt')
elif corr_type == 'pqtl':
    print(pqtl)
elif corr_type == 'sitesampleage':
    print('correct for site and sample age')
elif corr_type == 'sitesampleageqt':
    print('correct for site and sample age and normalize on training data')
elif corr_type == 'batch':
    print('correct for batch')
else:
    print('Normal version(just log transformed)')

### Logistic regression main models

In [ ]:
# K = [4, 9]
K = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]
try: 
    file = open(folder+pred_folder+"{}_{}_predict.pkl".format(endpoint,dataset),'rb')
    pred_dict = pickle.load(file)
except:
    pred_dict = {}

try: 
    file = open(folder+pred_folder + "{}_{}_test_prediction.pkl".format(endpoint,dataset),'rb')
    pred_test_dict = pickle.load(file)
except:
    print('No test predictions')
    pred_test_dict = {}    

### Run all predictions 
    
# model_keys =pred_dict.keys()
# # new_keys = []
# # for key in model_keys:
# #     if 'baseline2' in key:
# #         new_keys.append(key)
# # model_keys = new_keys
# for model_key in model_keys:
#     model = pred_dict[model_key][0]
#     feat = pred_dict[model_key][3]
# #     model = pred_dict['{}_proteinL1_y{}'.format(dataset,k)][0]
#     pred_test = model.predict_proba(X_test[feat])[:,1]
# #             pred_test =  pf.predictions_surv(model,feat,X_test,k)

#     pred_test_dict[model_key] = pred_test

#### Run selected predicitons

for k in K:
#     model_keys = ['{}_y{}_baselineminusMI_lr'.format(dataset,k)]   
#     model_keys = ['{}_y{}_baselineplusPCE_lr'.format(dataset,k)]   
#     model_keys = ['{}_y{}_agesexproteinqt_l1'.format(dataset,k),'{}_y{}_agesexGDF15qt_lr'.format(dataset,k)]
#     model_keys = ['{}_y{}_agesexproteincorr_l1'.format(dataset,k),'{}_y{}_agesexGDF15corr_lr'.format(dataset,k)]
#     model_keys = ['{}_y{}_agesexproteincorrqt_l1'.format(dataset,k),'{}_y{}_agesexGDF15corrqt_lr'.format(dataset,k)]
#     model_keys = ['{}_y{}_agesexproteinbatchcorr_l1'.format(dataset,k),'{}_y{}_agesexGDF15bathccorr_lr'.format(dataset,k)]
#     model_keys = ['{}_y{}_agesexproteinnoGDF15_l1'.format(dataset,k),'{}_y{}_proteinnoGDF15_l1'.format(dataset,k),'{}_y{}_baselineproteinnoGDF15_l1'.format(dataset,k)]
#     model_keys = ['{}_y{}_baseline2elife_lr'.format(dataset,k),'{}_y{}_baseline2elife_l2'.format(dataset,k)]
#     model_keys = ['{}_y{}_agesexprotein3_l1'.format(dataset,k),'{}_y{}_agesexprotein7_l1'.format(dataset,k)]
#     model_keys = ['{}_y{}_lifestyle_lr'.format(dataset,k),'{}_y{}_lifestyleGDF15_lr'.format(dataset,k)]
#     model_keys = ['{}_y{}_agesexPLAUR_lr'.format(dataset,k),'{}_y{}_baseline2PLAUR_lr'.format(dataset,k)]
#     model_keys = ['{}_y{}_GDF15_lr'.format(dataset,k)]
#     model_keys = ['{}_y{}_agesexstepBIC_l2'.format(dataset,k)]   
#             model_keys = ['{}_y{}_agesexGDF15ultra_lr'.format(dataset,k)]   
#             model_keys = ['{}_y{}_baselineGDF15PAD_lr'.format(dataset,k)]   
    model_keys = ['{}_y{}_baseline2PAD2_lr'.format(dataset,k)]     
#             model_keys = ['{}_y{}_agesexPAD_lr'.format(dataset,k)]       
#             model_keys = ['{}_y{}_protein_l1'.format(dataset,k)]
#             model_keys = ['{}_y{}_agesexproteinpqtl_l1'.format(dataset,k)]
#             model_keys = ['{}_y{}_baselineproteinPC{}_l1'.format(dataset,k,skip_PC)]
#             model_keys = ['{}_y{}_agesexproteinPC{}_l1'.format(dataset,k,skip_PC)]
    for model_key in model_keys:
        model = pred_dict[model_key][0]
        feat = pred_dict[model_key][3]
#     model = pred_dict['{}_proteinL1_y{}'.format(dataset,k)][0]
        pred_test = model.predict_proba(X_test[feat])[:,1]
#             pred_test =  pf.predictions_surv(model,feat,X_test,k)

        pred_test_dict[model_key] = pred_test

###################
#     f = open(folder+pred_folder+"{}_{}_test_prediction.pkl".format(endpoint,dataset),"wb")
#     pickle.dump(pred_test_dict,f)
#     f.close()
    print('Done')


In [ ]:

# f = open(folder+pred_folder+"{}_{}_test_prediction.pkl".format(endpoint,dataset),"wb")
# pickle.dump(pred_test_dict,f)
# f.close()
# print('Done')

pred_test_dict.keys()

In [ ]:
# pd.DataFrame(pred_dict['{}_y{}_baseline2hopro_lr'.format(dataset,9)][0].coef_[0],pred_dict['{}_y{}_baseline2hopro_lr'.format(dataset,9)][3])
# pred_dict['{}_y{}_baseline2hopro_lr'.format(dataset,9)][3],

### Cox models

In [ ]:

try: 
    file = open(folder+pred_folder+"{}_{}_predict_cox.pkl".format(endpoint,dataset),'rb')
    pred_dict = pickle.load(file)
except:
    pred_dict = {}

try: 
    file = open(folder+pred_folder + "{}_{}_test_prediction.pkl".format(endpoint,dataset),'rb')
    pred_test_dict = pickle.load(file)
except:
    print('No test predictions')
    pred_test_dict = {}    


K =[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]
for k in K:

#     model_key = '{}_boruta_y9_coxl2'.format(dataset)
    model_keys = pred_dict.keys()
    print(model_keys)
#     model_keys = ['{}_y{}_agesexstepBIC_l2'.format(dataset,k)]   
    for model_key in model_keys:
        model = pred_dict[model_key][0]
        feat = pred_dict[model_key][3]

        pred_test =  pf.predictions_surv(model,feat,X_test,k)

        pred_test_dict['predy{}_'.format(k)+ model_key] = np.array(pred_test[0])

#         f = open(folder+pred_folder+"{}_{}_test_prediction.pkl".format(endpoint,dataset),"wb")
#         pickle.dump(pred_test_dict,f)
#         f.close()
        print('Done')


In [ ]:
pred_test_dict.keys()

In [ ]:
# f = open(folder+pred_folder+"{}_{}_test_prediction.pkl".format(endpoint,dataset),"wb")
# pickle.dump(pred_test_dict,f)
# f.close()

### Predict single protein

In [ ]:
# K = [4, 9]
# K = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]
K=[0]
try: 
    file = open(folder+pred_folder+"{}_{}_predict_single_protein.pkl".format(endpoint,dataset),'rb')
    pred_dict = pickle.load(file)
except:
    pred_dict = {}

try: 
    file = open(folder+pred_folder + "{}_{}_test_prediction_single_protein.pkl".format(endpoint,dataset),'rb')
    pred_test_dict = pickle.load(file)
except:
    print('No test predictions')
    pred_test_dict = {}    
print(pred_dict.keys())
print("")
print(pred_test_dict.keys())

In [ ]:
# for k in K:

model_keys = pred_dict.keys()
for model_key in model_keys:
    model = pred_dict[model_key][0]
    feat = pred_dict[model_key][3]
#     model = pred_dict['{}_proteinL1_y{}'.format(dataset,k)][0]
    pred_test = model.predict_proba(X_test[feat])[:,1]
#             pred_test =  pf.predictions_surv(model,feat,X_test,k)

    pred_test_dict[model_key] = pred_test

f = open(folder+pred_folder+"{}_{}_test_prediction_single_protein.pkl".format(endpoint,dataset),"wb")
pickle.dump(pred_test_dict,f)
f.close()
print('Done')

In [ ]:
pred_test_dict.keys()